In [ ]:
!pip install boto3 awscli


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 875.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15

In [ ]:
import getpass

# Input AWS credentials and region
aws_access_key_id = getpass.getpass('Enter AWS Access Key ID: ')
aws_secret_access_key = getpass.getpass('Enter AWS Secret Access Key: ')
aws_region = input('Enter AWS Region: ')

!aws configure set aws_access_key_id $aws_access_key_id
!aws configure set aws_secret_access_key $aws_secret_access_key
!aws configure set default.region $aws_region

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········
Enter AWS Region: us-east-1


In [ ]:
stream_name = 'Fxweatherstream'


In [ ]:
!aws kinesis create-stream --stream-name Fxweatherstream


In [ ]:
!aws kinesis describe-stream-summary --stream-name Fxweatherstream

{
    "StreamDescriptionSummary": {
        "StreamName": "Fxweatherstream",
        "StreamARN": "arn:aws:kinesis:us-east-1:522723766177:stream/Fxweatherstream",
        "StreamStatus": "ACTIVE",
        "StreamModeDetails": {
            "StreamMode": "ON_DEMAND"
        },
        "RetentionPeriodHours": 24,
        "StreamCreationTimestamp": 1704024709.0,
        "EnhancedMonitoring": [
            {
                "ShardLevelMetrics": []
            }
        ],
        "EncryptionType": "NONE",
        "OpenShardCount": 4,
        "ConsumerCount": 0
    }
}


In [ ]:
import boto3
import json
import time
import random
# Initialize Kinesis client
kinesis_client = boto3.client('kinesis', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)


In [ ]:
def put_weather_data(city, temperature, humidity):
    try:
        data = {
            "city": city,
            "temperature": temperature,
            "humidity": humidity,
            "timestamp": int(time.time())
        }

        response = kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey=city
        )
        print(f"Weather data written to Kinesis. SequenceNumber: {response['SequenceNumber']}, ShardId: {response['ShardId']}, Data: {data}")
    except Exception as e:
        print(f"Error: {e}")


In [ ]:
if __name__ == "__main__":
    cities = ["New York", "Los Angeles", "Chicago", "Houston", "Miami"]

    for _ in range(5):
        city = random.choice(cities)
        temperature = round(random.uniform(10, 30), 2)
        humidity = round(random.uniform(40, 80), 2)

        put_weather_data(city, temperature, humidity)
        time.sleep(2)  # Simulating periodic data transmission

Weather data written to Kinesis. SequenceNumber: 49647873640263458610754731919468965264146644284999729202, ShardId: shardId-000000000003, Data: {'city': 'Los Angeles', 'temperature': 19.4, 'humidity': 47.03, 'timestamp': 1704025822}
Weather data written to Kinesis. SequenceNumber: 49647873640263458610754731919470174189966258982893912114, ShardId: shardId-000000000003, Data: {'city': 'Los Angeles', 'temperature': 17.46, 'humidity': 65.6, 'timestamp': 1704025825}
Weather data written to Kinesis. SequenceNumber: 49647873640263458610754731919471383115785873818227048498, ShardId: shardId-000000000003, Data: {'city': 'Los Angeles', 'temperature': 22.99, 'humidity': 79.78, 'timestamp': 1704025827}
Weather data written to Kinesis. SequenceNumber: 49647873640241157865556201378510206763276487563616976930, ShardId: shardId-000000000002, Data: {'city': 'Houston', 'temperature': 29.41, 'humidity': 58.55, 'timestamp': 1704025829}
Weather data written to Kinesis. SequenceNumber: 496478736402634586107